In [1]:
import pollux
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [40]:
latent_dim = 3
mu = Input((latent_dim,))
sigma = Input((latent_dim,))
cov = Input((latent_dim,latent_dim,))
# z, Dkl = pollux.layers.SampleMultivariateGaussian(full_cov=False, add_KL=True, return_KL=True)([mu, sigma])
z, Dkl = pollux.layers.SampleMultivariateGaussian(full_cov=True, add_KL=True, return_KL=True)([mu, cov])

[TensorShape([Dimension(None), Dimension(3)]), TensorShape([Dimension(None), Dimension(3), Dimension(3)])]
3


In [41]:
z

<tf.Tensor 'sample_multivariate_gaussian_6/MultivariateNormalFullCovariance_1/sample/affine_linear_operator/forward/add:0' shape=(?, 3) dtype=float32>

In [42]:
Dkl

<tf.Tensor 'sample_multivariate_gaussian_6/strided_slice:0' shape=(?, 1) dtype=float32>

In [44]:
# model = Model([mu, sigma], z)
# model2 = Model([mu, sigma], Dkl)

model = Model([mu, cov], z)
model2 = Model([mu, cov], Dkl)

In [45]:
def loss(x,y):
    return 0*tf.ones_like(x)

In [46]:
model.compile(loss=loss, optimizer='adam')
model2.compile(loss=loss, optimizer='adam')

In [47]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 3)            0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 3, 3)         0                                            
__________________________________________________________________________________________________
sample_multivariate_gaussian_6  [(None, 3), (None, 1 0           input_16[0][0]                   
                                                                 input_18[0][0]                   
Total params: 0
Trainable params: 0
Non-trainable params: 0
__________________________________________________________________________________________________


In [48]:
model.output

<tf.Tensor 'sample_multivariate_gaussian_6/MultivariateNormalFullCovariance_1/sample/affine_linear_operator/forward/add:0' shape=(?, 3) dtype=float32>

In [49]:
n = 10
mu_test = 0.*np.ones((n,latent_dim))
sigma_test = 1.*np.ones((n,latent_dim))
model.evaluate(x=[mu_test, sigma_test],y=0.*np.ones(n))

ValueError: Error when checking input: expected input_18 to have 3 dimensions, but got array with shape (10, 3)

In [33]:
n = 10
model.predict(x=[mu_test, sigma_test])

array([[ 0.8073305 ,  0.46063012,  1.3483598 ],
       [-0.3833886 , -1.7930347 ,  0.04047422],
       [ 0.15214176,  1.2053874 ,  0.09220563],
       [ 0.6939067 , -0.16165979,  1.491137  ],
       [ 1.447813  , -0.37499973, -0.3917515 ],
       [ 0.53756297, -0.5191131 ,  1.0345051 ],
       [-1.8660831 , -0.06141957,  1.6810693 ],
       [-1.3099229 , -0.06025485,  0.26224387],
       [-0.23380795, -1.3279732 ,  0.33207968],
       [-0.169808  ,  0.24748498, -1.2296381 ]], dtype=float32)

In [34]:
model2.predict(x=[mu_test,sigma_test])

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)